<a href="https://colab.research.google.com/github/jeffdi/tt_ml_test/blob/main/Pytorch_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An example implementation of a simple neural network using PyTorch and Tiny Tapeout

In this notebook, we are going to train a simple neural network using PyTorch and implement it in silicon using Tiny Tapeout.

---

**Author:** TBD

## Importing the libraries

Like with any other programming exercise, the first step is to import the necessary libraries. As we are going to be using Google Colab to program our neural network, we need to install and import the necessary PyTorch libraries.

In [ ]:
!git clone https://github.com/rohittp0/chipon
%cd chipon/

In [19]:
import torch
import torch.nn as nn

import numpy as np

import layers
from constants import test_bench_template
from main import Model

## print out the pytorch version used
print(torch.__version__)

2.2.1+cu121


## The Neural Network

![alt text](https://drive.google.com/uc?export=view&id=1Lpi4VPBfAV3JkOLopcsGK4L8dyxmPF1b)

Before building and training a neural network the first step is to process and prepare the data. In this notebook, we are going to use syntethic data (i.e., fake data) so we won't be using any real world data.

For the sake of simplicity, we are going to use the following input and output pairs converted to tensors, which is how data is typically represented in the world of deep learning. The x values represent the input of dimension `(6,1)` and the y values represent the output of similar dimension. The example is taken from this [tutorial](https://github.com/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%202%20-%20Lesson%202%20-%20Notebook.ipynb).

The objective of the neural network model that we are going to build and train is to automatically learn patterns that better characterize the relationship between the `x` and `y` values. Essentially, the model learns the relationship that exists between inputs and outputs which can then be used to predict the corresponding `y` value for any given input `x`.

In [15]:
simple_model = nn.Sequential(
        nn.Linear(2, 1),
        nn.ReLU(),
    )

In [16]:
simple_model[0].weight = nn.Parameter(torch.tensor([[1.0, -1.0]]))
simple_model[0].bias = nn.Parameter(torch.tensor([1.0]))

In [20]:
model = Model(simple_model)
model.parse_layers()
model.forward_range([[1.0, 100.0], [0.0, 1024.0]])

In [21]:
print(model)
code = model.emit()

Linear(2 -> 1)
ReLU((1,))


In [23]:
print(code)

`timescale 1ns / 1ps

module layer_0_linear_2_1(in0,in1, out0);
    input [6:0] in0;
    input [10:0] in1;

    output [9:0] out0;

    
    reg [9:0] mul0;

    reg [9:0] add0;

        
    always @(*)
    begin
        mul0 = mul0 + in0 * 1.0;
        mul0 = mul0 + in1 * -1.0;

        add0 = mul0 + 1.0;

    end
    assign out0 = add0;

endmodule


module layer_1_relu_1(in0, out0);
    input [9:0] in0;

    output reg [1:0] out0;

                
    always @(*) 
    begin
        out0 = in0 > 0 ? in0 : 0;

    end
endmodule

module top(in0,in1, out0);
    input [6:0] in0;
    input [10:0] in1;
    output [1:0] out0;
    wire [10:0] layer_0_out_0;
    layer_0_linear_2_1 layer_0(in0,in1, layer_0_out_0);
    wire [2:0] layer_1_out_0;
    layer_1_relu_1 layer_1(layer_0_out_0, layer_1_out_0);
    assign out0 = layer_1_out_0;
endmodule


In [22]:
with open('test.v', 'w') as f:
    f.write(code)

with open('test_tb.v', 'w') as f:
    f.write(model.emit_test_bench())

In [ ]:
## our data in tensor form
x = torch.tensor([[-1.0],  [0.0], [1.0], [2.0], [3.0], [4.0]], dtype=torch.float)
y = torch.tensor([[-3.0], [-1.0], [1.0], [3.0], [5.0], [7.0]], dtype=torch.float)

In [ ]:
## print size of the input tensor
x.size()

torch.Size([6, 1])

## The Neural Network Components
As said earlier, we are going to first define and build out the components of our neural network before training the model.

### Model

Typically, when building a neural network model, we define the layers and weights which form the basic components of the model. Below we show an example of how to define a hidden layer named `layer1` with size `(1, 1)`. For the purpose of this tutorial, we won't explicitly define the `weights` and allow the built-in functions provided by PyTorch to handle that part for us. By the way, the `nn.Linear(...)` function applies a linear transformation ($y = xA^T + b$) to the data that was provided as its input. We ignore the bias for now by setting `bias=False`.





In [ ]:
## Neural network with 1 hidden layer
layer1 = nn.Linear(1,1, bias=False)
model = nn.Sequential(layer1)

### Loss and Optimizer
The loss function, `nn.MSELoss()`, is in charge of letting the model know how good it has learned the relationship between the input and output. The optimizer (in this case an `SGD`) primary role is to minimize or lower that loss value as it tunes its weights.

In [ ]:
## loss function
criterion = nn.MSELoss()

## optimizer algorithm
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

## Training the Neural Network Model
We have all the components we need to train our model. Below is the code used to train our model.

In simple terms, we train the model by feeding it the input and output pairs for a couple of rounds (i.e., `epoch`). After a series of forward and backward steps, the model somewhat learns the relationship between x and y values. This is notable by the decrease in the computed `loss`. For a more detailed explanation of this code check out this [tutorial](https://medium.com/dair-ai/a-simple-neural-network-from-scratch-with-pytorch-and-google-colab-c7f3830618e0).

In [ ]:
## training
for ITER in range(150):
    model = model.train()

    ## forward
    output = model(x)
    loss = criterion(output, y)
    optimizer.zero_grad()

    ## backward + update model params
    loss.backward()
    optimizer.step()

    model.eval()
    print('Epoch: %d | Loss: %.4f' %(ITER, loss.detach().item()))

Epoch: 0 | Loss: 5.4530
Epoch: 1 | Loss: 4.4949
Epoch: 2 | Loss: 3.7246
Epoch: 3 | Loss: 3.1053
Epoch: 4 | Loss: 2.6073
Epoch: 5 | Loss: 2.2069
Epoch: 6 | Loss: 1.8850
Epoch: 7 | Loss: 1.6262
Epoch: 8 | Loss: 1.4181
Epoch: 9 | Loss: 1.2508
Epoch: 10 | Loss: 1.1163
Epoch: 11 | Loss: 1.0082
Epoch: 12 | Loss: 0.9212
Epoch: 13 | Loss: 0.8513
Epoch: 14 | Loss: 0.7951
Epoch: 15 | Loss: 0.7499
Epoch: 16 | Loss: 0.7136
Epoch: 17 | Loss: 0.6844
Epoch: 18 | Loss: 0.6609
Epoch: 19 | Loss: 0.6420
Epoch: 20 | Loss: 0.6268
Epoch: 21 | Loss: 0.6146
Epoch: 22 | Loss: 0.6048
Epoch: 23 | Loss: 0.5969
Epoch: 24 | Loss: 0.5905
Epoch: 25 | Loss: 0.5854
Epoch: 26 | Loss: 0.5813
Epoch: 27 | Loss: 0.5780
Epoch: 28 | Loss: 0.5754
Epoch: 29 | Loss: 0.5733
Epoch: 30 | Loss: 0.5715
Epoch: 31 | Loss: 0.5702
Epoch: 32 | Loss: 0.5691
Epoch: 33 | Loss: 0.5682
Epoch: 34 | Loss: 0.5675
Epoch: 35 | Loss: 0.5669
Epoch: 36 | Loss: 0.5664
Epoch: 37 | Loss: 0.5660
Epoch: 38 | Loss: 0.5657
Epoch: 39 | Loss: 0.5655
Epoch: 40 

## Testing the Model
After training the model we have the ability to test the model predictive capability by passing it an input. Below is a simple example of how you could achieve this with our model. The result we obtained aligns with the results obtained in this [notebook](https://github.com/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%202%20-%20Lesson%202%20-%20Notebook.ipynb), which inspired this entire tutorial.

In [ ]:
## test the model
sample = torch.tensor([10.0], dtype=torch.float)
predicted = model(sample)
print(predicted.detach().item())

17.096769332885742


## Final Words

Congratulations! In this tutorial you learned how to train a simple neural network using PyTorch. You also learned about the basic components that make up a neural network model such as the linear transformation layer, optimizer, and loss function. We then trained the model and tested its predictive capabilities. You are well on your way to become more knowledgeable about deep learning and PyTorch. I have provided a bunch of references below if you are interested in practising and learning more.

*I would like to thank Laurence Moroney for his excellent [tutorial](https://github.com/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%202%20-%20Lesson%202%20-%20Notebook.ipynb) which I used as an inspiration for this tutorial.*

## Exercises
- Add more examples in the input and output tensors. In addition, try to change the dimensions of the data, say by adding an extra value in each array. What needs to be changed to successfully train the network with the new data?
- The model converged really fast, which means it learned the relationship between x and y values after a couple of iterations. Do you think it makes sense to continue training? How would you automate the process of stopping the training after the model loss doesn't subtantially change?
- In our example, we used a single hidden layer. Try to take a look at the PyTorch documentation to figure out what you need to do to get a model with more layers. What happens if you add more hidden layers?
- We did not discuss the learning rate (`lr-0.001`) and the optimizer in great detail. Check out the [PyTorch documentation](https://pytorch.org/docs/stable/optim.html) to learn more about what other optimizers you can use.


## References
- [The Hello World of Deep Learning with Neural Networks](https://github.com/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%202%20-%20Lesson%202%20-%20Notebook.ipynb)
- [A Simple Neural Network from Scratch with PyTorch and Google Colab](https://medium.com/dair-ai/a-simple-neural-network-from-scratch-with-pytorch-and-google-colab-c7f3830618e0?source=collection_category---4------1-----------------------)
- [PyTorch Official Docs](https://pytorch.org/docs/stable/nn.html)
- [PyTorch 1.2 Quickstart with Google Colab](https://medium.com/dair-ai/pytorch-1-2-quickstart-with-google-colab-6690a30c38d)
- [A Gentle Intoduction to PyTorch](https://medium.com/dair-ai/pytorch-1-2-introduction-guide-f6fa9bb7597c)